Este trabalho é referente à disciplina de Inteligência Artificial do curso de Bacharelado em Ciência da Computação na Universidade Federal do Pará. Ele propõe a implementação de uma versão do algoritmo Stochastic Hill Climbing (Simulated Annealing) e de uma meta-heurística (Genetic Algorithm) para a resolução do problema das oito rainhas.

Durante todo o ciclo de desenvolvimento deste trabalho, o GitHub foi utilizado como ferramenta de versionamento. O histórico de desenvolvimento da equipe, bem como versões iniciais do código aqui contido, está disponível ao público no GitHub em [@ygarasab/kmeans](https://github.com/ygarasab/kmeans).

**Observação:** antes de interagir com o código no Jupyter Notebook/Lab, é importante observar que certas células anteriores à que se quer executar podem ser necessárias. A fim de evitar isso, é importante certificar-se de executar ao menos uma vez todas as células contendo definições de funções ou importações de módulos.

# Sumário

1. [Requisitos](#Requisitos)
    1. [Ambiente](#Ambiente)
    2. [Funções de checagem](#Funções-de-checagem)
2. [Modelagem do algoritmo, da função-objetivo e das soluções](#Modelagem-do-algoritmo,-da-função-objetivo-e-das-soluções)
    1. [Funções auxiliares](#Funções-auxiliares)
    2. [Funções auxiliares relacionadas ao K-Means](#Funções-auxiliares-relacionadas-ao-K-Means)
    3. [A classe `KMedias`](#A-classe-KMedias)
    4. [A função-objetivo](#A-função-objetivo)
    5. [As soluções](#As-soluções)
3. [O problema](#O-problema)
4. [O conjunto de dados](#O-conjunto-de-dados)
    1. [Pré-processamento](#Pré-processamento)

# Requisitos

## Ambiente

Este trabalho foi feito utilizando a seguinte linguagem de programação:

- `python` 3.8.5

Em adição, os seguintes módulos precisam estar instalados no ambiente em que este notebook for executado:

- `ipypublish` 0.10.12
- `matplotlib` 3.3.2
- `numba` 0.52.0
- `numpy` 1.19.2
- `pandas` 1.1.4
- `scipy` 1.5.3
- `seaborn` 0.11.0

Observe que o código pode funcionar em versões distintas às que foram mencionadas acima; contudo, sabe-se que a sua execução é garantida nas versões mencionadas.

In [1]:
import decimal as d
import numba as nb
import numpy as np
import random as r
import seaborn as sns
import typing as t

from ipypublish import nb_setup
from scipy import stats
from timeit import default_timer as timer

In [2]:
%%capture

plt = nb_setup.setup_matplotlib(output=('pdf',), usetex=False, rcparams={'axes.facecolor': 'white', 'figure.facecolor': 'white'})
pd = nb_setup.setup_pandas(escape_latex=True)

Antes de começarmos, é importante mencionar que buscou-se aqui, sempre que possível, utilizar funções `numba`-jitted de forma a compilar as funções redigidas em `python` em código de máquina. Dessa forma, busca-se maximizar o desempenho da implementação desenvolvida.

## Funções de checagem

Uma vez configurado o ambiente em que este notebook será executado, é necessário definir algumas funções de checagem que serão utilizadas mais adiante:

In [3]:
def verifica_comprimento_igual_a(**parametros):
    numero_de_parametros = len(parametros.keys())

    if numero_de_parametros != 2:
        raise ValueError(
            f"Apenas um parâmetro pode ser passado para esta função. Foram recebidos " f"{numero_de_parametros}."
        )

    parametro, outro_parametro = parametros.keys()

    valor, descricao = parametros[parametro]
    outro_valor, outra_descricao = parametros[outro_parametro]

    if outro_valor is not None and len(valor) != len(outro_valor):
        raise ValueError(
            f"O {descricao} {parametro} precisa ter um comprimento igual ao {outra_descricao} " f"{outro_parametro}."
        )

In [4]:
def verifica_comprimento_menor_ou_igual_a(**parametros):
    numero_de_parametros = len(parametros.keys())

    if numero_de_parametros != 2:
        raise ValueError(
            f"Apenas um parâmetro pode ser passado para esta função. Foram recebidos " f"{numero_de_parametros}."
        )

    parametro, outro_parametro = parametros.keys()

    valor, descricao = parametros[parametro]
    outro_valor, outra_descricao = parametros[outro_parametro]

    if outro_valor is not None and len(valor) > len(outro_valor):
        raise ValueError(
            f"O {descricao} {parametro} precisa ter um comprimento, no máximo, igual ao {outra_descricao} "
            f"{outro_parametro}."
        )

In [5]:
def verifica_dtype(**parametro_dict):
    numero_de_parametros = len(parametro_dict.keys())

    if numero_de_parametros != 1:
        raise ValueError(
            f"Apenas um parâmetro pode ser passado para esta função. Foram recebidos " f"{numero_de_parametros}."
        )

    parametro = list(parametro_dict.keys())[0]
    valor, descricao, dtype = parametro_dict[parametro]

    if dtype == np.int_ and valor.dtype != dtype:
        if valor.dtype == np.float_:
            return valor.astype(np.int_)
        else:
            raise TypeError(
                f"O {descricao} {parametro} precisa ser um numpy array com atributo dtype igual a "
                f"{dtype}. O dtype do numpy array recebido é {valor.dtype}."
            )

    if valor.dtype != dtype:
        raise TypeError(
            f"O {descricao} {parametro} precisa ser um numpy array com atributo dtype igual a {dtype}. "
            f"O dtype do numpy array recebido é {valor.dtype}."
        )
    else:
        return valor

In [6]:
def verifica_nao_negatividade(**parametros):
    for parametro in parametros.keys():
        valor, descricao = parametros[parametro]

        if valor < 0:
            raise ValueError(f"O {descricao} {parametro} precisa receber um número não-negativo.")

In [7]:
def verifica_ndim(**parametros):
    for parametro in parametros.keys():
        valor, descricao, ndim = parametros[parametro]

        if valor.ndim != ndim:
            raise ValueError(f"O o atributo ndim do {descricao} {parametro} precisa ser igual a {ndim}.")

In [8]:
def verifica_tipo(**parametro_dict):
    numero_de_parametros = len(parametro_dict.keys())

    if numero_de_parametros != 1:
        raise ValueError(
            f"Apenas um parâmetro pode ser passado para esta função. Foram recebidos " f"{numero_de_parametros}."
        )

    parametro = list(parametro_dict.keys())[0]
    valor, descricao, tipos = parametro_dict[parametro]

    if tipos == t.SupportsFloat:
        if not isinstance(valor, tipos):
            raise TypeError(
                f"O {descricao} {parametro} precisa receber um número de ponto flutuante ou um objeto que "
                f"possa ser convertido para tal."
            )
        else:
            return float(valor)

    if tipos == t.SupportsInt:
        if not isinstance(valor, tipos):
            raise TypeError(
                f"O {descricao} {parametro} precisa receber um número inteiro ou um objeto que possa ser "
                f"convertido para tal."
            )
        else:
            return int(valor)

    if tipos == np.ndarray:
        if not isinstance(valor, np.ndarray):
            if not isinstance(valor, (list, tuple)):
                raise TypeError(
                    f"O {descricao} {parametro} precisa receber um array numpy ou um objeto que possa ser "
                    f"convertido para tal."
                )
            else:
                return np.array(valor)

    if tipos == bool:
        if not isinstance(valor, bool):
            if isinstance(valor, np.bool_):
                return bool(valor)
            else:
                raise TypeError(
                    f"O {descricao} {parametro} precisa receber um objeto booleano ou um objeto que possa "
                    f"ser convertido para tal."
                )
        else:
            return valor

    if not isinstance(valor, tipos):
        raise TypeError(f"O {descricao} {parametro} precisa receber um objeto de classe {tipos} ou que herde dela.")
    else:
        return valor

# Modelagem do algoritmo, da função-objetivo e das soluções

Em aderência ao estipulado pelo comando do trabalho, o algoritmo K-Means foi escolhido como o clusterizador para a resolução do problema dos aeroportos. Antes de definirmos a classe `KMedias`, no entanto, algumas funções hão de serem definidas para auxiliar no desempenhar das atribuições do algoritmo. Embora estas funções sejam sabidamente já implementadas na forma de métodos e funções do módulo `numpy`, elas não foram implementadas em código compilável pelo módulo `numba`, o que faz com que o processo de tradução do módulo `llvmlite` falhe por não reconhecer a função ou método em questão. Dessa forma, as implementações relevevantes e ausentes foram feitas manualmente, também utilizando o módulo `numba`.

## Funções auxiliares

In [9]:
@nb.jit(nopython=True, parallel=True)
def arredonda_matriz(matriz, casas_decimais):
    vetor = matriz.copy().ravel()
    comprimento = vetor.shape[0]

    for i in nb.prange(comprimento):
        vetor[i] = np.round_(vetor[i], casas_decimais)

    matriz_arredondada = vetor.reshape(matriz.shape)

    return matriz_arredondada

In [10]:
@nb.jit(nopython=True, parallel=True)
def calcula_distancia_entre_grupos(grupo_um, grupo_dois, tira_raiz=True):
    comprimento_um, comprimento_dois = grupo_um.shape[0], grupo_dois.shape[0]
    distancias = np.empty(shape=(comprimento_um, comprimento_dois))

    for i_um in nb.prange(comprimento_um):
        for i_dois in nb.prange(comprimento_dois):
            distancias[i_um, i_dois] = calcula_distancia_entre_observacoes(
                grupo_um[i_um, :], grupo_dois[i_dois, :], tira_raiz
            )

    return distancias

In [11]:
@nb.jit(nopython=True, parallel=True)
def calcula_distancia_entre_observacoes(vetor_um, vetor_dois, tira_raiz):
    comprimento = vetor_um.shape[0]
    diferencas = np.empty(shape=comprimento)

    for i in nb.prange(comprimento):
        diferencas[i] = np.square(vetor_um[i] - vetor_dois[i])

    distancia = np.sum(diferencas)

    if tira_raiz is True:
        distancia = np.sqrt(distancia)

    return distancia

In [12]:
@nb.jit(nopython=True)
def verifica_igualdade_aproximada_entre_grupos(grupo_um, grupo_dois, casas_decimais):
    grupo_um, grupo_dois = arredonda_matriz(grupo_um, casas_decimais), arredonda_matriz(grupo_dois, casas_decimais)
    ha_igualdade = bool((grupo_um == grupo_dois).all())

    return ha_igualdade

In [13]:
@nb.jit(nopython=True, parallel=True)
def tira_media_das_colunas(dados):
    dimensionalidade = dados.shape[1]
    media_das_colunas_dos_dados = np.empty(shape=dimensionalidade)

    for f in nb.prange(dimensionalidade):
        media_das_colunas_dos_dados[f] = dados[:, f].mean()

    return media_das_colunas_dos_dados

## Funções auxiliares relacionadas ao K-Means

Uma vez definidas as funções auxiliares de propósito geral, definamos agora as funções auxiliares relacionadas à classe `KMedias`.

In [14]:
@nb.jit(nopython=True, parallel=True)
def calcula_erro_da_solucao(dados, centroides):
    rotulos, numero_de_centroides = rotula_dados(dados, centroides), centroides.shape[0]
    erro_dos_agrupamentos = np.empty(numero_de_centroides)

    for rotulo in nb.prange(numero_de_centroides):
        membros_do_agrupamento = np.where(rotulos == rotulo)[0]

        if membros_do_agrupamento.shape[0] == 0:
            return np.inf
        else:
            centroide = centroides[rotulo, :].reshape(1, -1)
            membros = dados[membros_do_agrupamento, :]
            erro_dos_agrupamentos[rotulo] = calcula_distancia_entre_grupos(
                centroide, membros, tira_raiz=False
            ).sum()

    erro_total = erro_dos_agrupamentos.sum()

    return erro_total

In [15]:
@nb.jit(nopython=True, parallel=True)
def centraliza_centroides(dados, centroides, centroides_fixos, rotulos):
    numero_de_centroides, dimensionalidade = centroides.shape
    rotulos_unicos = np.arange(numero_de_centroides)[np.logical_not(centroides_fixos)]
    numero_de_rotulos = rotulos_unicos.shape[0]
    centroides_centralizados = np.empty((numero_de_centroides, dimensionalidade))
    centroides_centralizados[centroides_fixos, :] = centroides[centroides_fixos, :]

    for r in nb.prange(numero_de_rotulos):
        rotulo = rotulos_unicos[r]
        membros_do_agrupamento = np.where(rotulos == rotulo)[0]
        agrupamento = dados[membros_do_agrupamento, :]
        centroides_centralizados[rotulo, :] = tira_media_das_colunas(agrupamento)

    return centroides_centralizados

In [16]:
@nb.jit(nopython=True)
def rotula_dados(dados, centroides):
    distancias = calcula_distancia_entre_grupos(dados, centroides)
    numero_de_observacoes = dados.shape[0]
    rotulos = np.empty(numero_de_observacoes, dtype=np.int_)

    for d in nb.prange(numero_de_observacoes):
        rotulos[d] = distancias[d].argmin()

    return rotulos

In [17]:
@nb.jit(nopython=True)
def gera_centroides(dados, numero_de_centroides):
    comprimento, dimensionalidade = dados.shape
    indices = np.arange(comprimento, dtype=np.int_)
    indices_aleatorios = np.random.choice(indices, size=numero_de_centroides, replace=False)
    centroides = dados[indices_aleatorios].copy()

    return centroides

In [18]:
@nb.jit(nopython=True)
def roda_k_means(dados, centroides, centroides_fixos, casas_decimais=4, numero_maximo_de_iteracoes=1000):
    iteracao = 0

    while iteracao < numero_maximo_de_iteracoes:
        rotulos = rotula_dados(dados, centroides)
        centroides_centralizados = centraliza_centroides(dados, centroides, centroides_fixos, rotulos)
        ha_igualdade = verifica_igualdade_aproximada_entre_grupos(
            centroides, centroides_centralizados, casas_decimais
        )
        centroides = centroides_centralizados

        if ha_igualdade is True:
            break
        else:
            iteracao += 1

    return centroides

### A classe `KMedias`

Uma vez definidas todas as funções necessárias, chegamos à definição da classe `KMedias`. Esta implementação do K-Means foi personalizada para suportar a especificação de centroides fixos, i.e., centroides que não são atualizados no decorrer das iterações, mas que compõem a solução final.

Em suma, esta classe é um *wrapper* para as funções `numba` supra-implementadas que serve para armazenar informações e realizar verificações nas entradas fornecidas aos seus métodos e atributos.

In [19]:
class KMedias:
    def __init__(self, *, numero_de_centroides):
        self.numero_de_centroides = numero_de_centroides
        self.__centroides, self.__centroides_fixos, self.__dados = None, None, None

    @property
    def numero_de_centroides(self):
        return self.__numero_de_centroides

    @numero_de_centroides.setter
    def numero_de_centroides(self, novo_numero_de_centroides):
        novo_numero_de_centroides = verifica_tipo(
            numero_de_centroides=(novo_numero_de_centroides, "atributo", t.SupportsInt)
        )

        verifica_nao_negatividade(numero_de_centroides=(novo_numero_de_centroides, "atributo"))

        self.__numero_de_centroides = novo_numero_de_centroides

    @property
    def centroides(self):
        return self.__centroides

    @centroides.setter
    def centroides(self, novos_centroides):
        if novos_centroides is None:
            novos_centroides = gera_centroides(
                dados=self.dados, numero_de_centroides=self.numero_de_centroides
            )
            self.centroides_fixos = novos_centroides.shape[0] * [False]
        elif isinstance(novos_centroides, np.ndarray):
            novos_centroides = verifica_tipo(centroides=(novos_centroides, "atributo", np.ndarray))

            verifica_ndim(centroides=(novos_centroides, "atributo", 2))
            verifica_comprimento_menor_ou_igual_a(
                centroides=(novos_centroides, "atributo"), dados=(self.dados, "atributo")
            )

            centroides_faltantes = self.numero_de_centroides - novos_centroides.shape[0]

            if centroides_faltantes > 0:
                centroides_complementares = gera_centroides(
                    dados=self.dados, numero_de_centroides=centroides_faltantes
                )
                self.centroides_fixos = novos_centroides.shape[0] * [True] + centroides_faltantes * [False]
                novos_centroides = np.concatenate((novos_centroides, centroides_complementares), axis=0)
            elif centroides_faltantes < 0:
                novos_centroides = novos_centroides[:centroides_faltantes]
                self.centroides_fixos = novos_centroides.shape[0] * [False]

        self.__centroides = novos_centroides

    @property
    def centroides_fixos(self):
        return self.__centroides_fixos

    @centroides_fixos.setter
    def centroides_fixos(self, novos_centroides_fixos):
        novos_centroides_fixos = verifica_tipo(
            centroides_fixos=(novos_centroides_fixos, "atributo", np.ndarray)
        )

        verifica_ndim(centroides_fixos=(novos_centroides_fixos, "atributo", 1))
        verifica_dtype(centroides_fixos=(novos_centroides_fixos, "atributo", np.bool_))
        verifica_comprimento_igual_a(
            centroides_fixos=(novos_centroides_fixos, "atributo"), centroides=(self.centroides, "atributo")
        )

        self.__centroides_fixos = novos_centroides_fixos

    @property
    def dados(self):
        return self.__dados

    @dados.setter
    def dados(self, novos_dados):
        novos_dados = verifica_tipo(dados=(novos_dados, "atributo", np.ndarray))

        verifica_ndim(dados=(novos_dados, "atributo", 2))

        self.__dados = novos_dados

    @property
    def rotulos(self):
        return rotula_dados(dados=self.dados, centroides=self.centroides)

    def clusteriza_dados(self, *, dados, centroides_fixos, numero_de_execucoes=1000):
        melhor_solucao, erro_da_melhor_solucao = None, None
        data_frame = pd.DataFrame(index=range(numero_de_execucoes), columns=["Solução", "Erro"])

        for iteracao in range(numero_de_execucoes):
            print(f"Execução {iteracao}...", end="\r")

            solucao = self._clusteriza_dados(dados=dados, centroides_fixos=centroides_fixos)
            erro_da_solucao = calcula_erro_da_solucao(dados, solucao)

            data_frame.iloc[iteracao, :] = np.array([solucao, erro_da_solucao], dtype=object)

            if melhor_solucao is None or erro_da_solucao < erro_da_melhor_solucao:
                melhor_solucao, erro_da_melhor_solucao = solucao, erro_da_solucao

        data_frame = data_frame.infer_objects().sort_values(by="Erro")
                
        return data_frame

    def _clusteriza_dados(self, *, dados, centroides_fixos):
        self.dados = dados
        self.centroides = centroides_fixos

        self.centroides = roda_k_means(
            dados=dados, centroides=self.centroides, centroides_fixos=self.centroides_fixos
        )

        return self.centroides

Pode-se notar que o método privado `KMedias._clusteriza_dados()`, que é chamado pelo método público `KMedias.clusteriza_dados()`, se vale da função `numba`-jitted `roda_k_means()`, que por sua vez chama as demais funções `numba`-jitted para desempenhar a rotina principal do K-Means.

## A função-objetivo

A função-objetivo a ser minimizada, doravante referida como erro, foi definida na função `numba`-jitted `calcula_erro_da_solucao()`. Ela segue à risca a função estipulada no trabalho, que é a que segue:

$$f(X, C)=\sum_{k=1}^{K} \sum_{\vec{x} \in \vec{C}_{i}}\left\|\vec{x}-\vec{C}_{i}\right\|^{2}$$

Nela, $X$ corresponde ao conjunto de dados, enquanto que $C$ corresponde ao conjunto de $K$ centroides. Busca-se, por meio dela, minimizar a distância das observações em relação ao respectivo centroide ao qual elas pertencem.

## As soluções

O algoritmo retorna um conjunto de $K$ centroides (onde $K$ é definido pelo usuário no parâmetro `numero_de_centroides` do construtor da classe `KMedias`) capaz de particionar o conjunto de dados fornecido em clusters.

# O problema

A partir do mencionado no arquivo `.pdf` contendo a descrição do trabalho, tem-se um cenário hipotético em que 3 novos aeroportos serão construídos na cidade Belém, localizada no estado Pará. Os seguintes distritos foram elencados para consideração:

- Distrito administrativo de Belém - Centro (DABEL)
- Distrito administrativo do Entroncamento (DAENT)
- Distrito administrativo do Guamá (DAGUA)
- Distrito administrativo do Benguí – Nova Belém (DABEN)
- Distrito administrativo da Sacramenta (DASAC)

A solução deve levar em conta o já existente Aeroporto Internacional Val-de-Cans que atende a região atualmente. A fim de descobrir soluções para esse problema, os seguintes componentes fazem-se necessários:

- **Um algoritmo de clusterização**, que deve ser executado várias vezes para assegurar que a melhor solução possível será encontrada;
- **Um conjunto de dados**, que deve descrever o centro dos bairros dos distritos considerados; e
- **As coordenadas do Aeroporto Internacional Val-de-Cans**, que deve ser considerado como um centroide fixo na solução a ser descoberta.

Em adição, os seguintes requisitos precisam ser atendidos:

- Cada novo aeroporto deve atender ao máximo de bairros próximos a ele; e
- Cada novo aeroporto deve atender a pelo menos um bairro.

# O conjunto de dados

Em seguimento às estipulações do problema proposto, os centros dos bairros dos distritos sob análise foram obtidos e armazenados no arquivo `coordenadas_bairros.csv`. Vamos carregar e visualizar um trecho de seu conteúdo.

In [20]:
aeroporto = pd.read_csv("dados/coordenadas_aeroporto.csv", index_col=0)
aeroporto

,Coordenadas
Aeroporto,
Val-de-Cans,"-1.382092, -48.477506"


In [21]:
aeroporto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, Val-de-Cans to Val-de-Cans
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Coordenadas  1 non-null      object
dtypes: object(1)
memory usage: 16.0+ bytes


In [22]:
bairros = pd.read_csv("dados/coordenadas_bairros.csv", index_col=0)
bairros.head(5)

,Coordenadas
Bairro,
Aeroporto,"-1.1423013, -48.4545478"
Água Boa,"-1.2579676, -48.4541267"
Águas Lindas,"-1.4106819, -48.3925318"
Águas Negras,"-1.3045941, -48.4432742"
Agulha,"-1.3049139, -48.4711298"


In [23]:
bairros.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71 entries, Aeroporto to Vila
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Coordenadas  71 non-null     object
dtypes: object(1)
memory usage: 1.1+ KB


## Pré-processamento

Como se pode ver, as coordenadas do aeroporto e dos bairros são descritas por objetos `str` e de forma conjunta. Em adição, os dados estão em um objeto `pandas.DataFrame`, quando a implementação realizada na classe `KMedias` suporta apenas objetos `numpy.ndarray`.

Dessa forma, precisamos

- separar as coordenadas em `Latitude` e `Longitude`; e
- converter os objetos `pandas.DataFrame` em objetos `numpy.ndarray`.

Comecemos pela definição das colunas `Latitude` e `Longitude`.

In [24]:
aeroporto["Latitude"] = aeroporto["Coordenadas"].apply(lambda x: x.split(", ")[0])
aeroporto["Longitude"] = aeroporto["Coordenadas"].apply(lambda x: x.split(", ")[1])

In [25]:
aeroporto.drop("Coordenadas", axis=1, inplace=True)

In [26]:
aeroporto["Latitude"] = pd.to_numeric(aeroporto["Latitude"])
aeroporto["Longitude"] = pd.to_numeric(aeroporto["Longitude"])

In [27]:
aeroporto

,Latitude,Longitude
Aeroporto,,
Val-de-Cans,-1.382092,-48.477506


In [28]:
bairros["Latitude"] = bairros["Coordenadas"].apply(lambda x: x.split(", ")[0])
bairros["Longitude"] = bairros["Coordenadas"].apply(lambda x: x.split(", ")[1])

In [29]:
bairros.drop("Coordenadas", axis=1, inplace=True)

In [30]:
bairros["Latitude"] = pd.to_numeric(bairros["Latitude"])
bairros["Longitude"] = pd.to_numeric(bairros["Longitude"])

In [31]:
bairros.head(5)

,Latitude,Longitude
Bairro,,
Aeroporto,-1.142301,-48.454548
Água Boa,-1.257968,-48.454127
Águas Lindas,-1.410682,-48.392532
Águas Negras,-1.304594,-48.443274
Agulha,-1.304914,-48.471130


Por fim, convertamos os objetos `pandas.DataFrame` em objetos `numpy.ndarray`.

In [32]:
aeroporto = aeroporto.to_numpy()
bairros = bairros.to_numpy()

In [33]:
aeroporto

array([[ -1.382092, -48.477506]])

In [34]:
bairros[:5, :]

array([[ -1.1423013, -48.4545478],
       [ -1.2579676, -48.4541267],
       [ -1.4106819, -48.3925318],
       [ -1.3045941, -48.4432742],
       [ -1.3049139, -48.4711298]])

## Processamento

Uma vez pré-processado o conjunto de dados, temos as variáveis `aeroporto`, que contém o centroide fixo representativo do Aeroporto Internacional Val-de-Cans, e `bairros`, que contém as coordenadas de todos os bairros relevantes. Já podemos seguir para o instanciamento e uso da classe `KMedias`; contudo, definamos antes algumas funções auxiliares para a exposição dos resultados e execução dos testes.